# Experiments for ClaimPT Resource Paper (generative)

## Install LangExtract and others

In [3]:
!pip install langextract
!pip install spacy
!pip install dkpro-cassis
!python -m spacy download pt_core_news_sm



[notice] A new release of pip is available: 23.2.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


## Load Data

In [7]:
from utils.parse_data import getData, convertData, convertDatatoPandas

In [6]:
"""
ARGUMENT to pass to getData is the folder from the exported project from inception in CAS (in this case is called curation)
"""
annotations=getData()
data=convertData(annotations)

In [8]:
df=convertDatatoPandas(annotations)

In [9]:
df

,label,begin,end,doc_id,text,token_text
0,None,0,74,1,Criado sistema de depósito e reembolso para em...,Criado sistema de depósito e reembolso para em...
1,None,85,96,1,Criado sistema de depósito e reembolso para em...,04 dez 2023
2,None,0,76,2,"Nova reitora da Universidade de Évora quer ""vo...","Nova reitora da Universidade de Évora quer ""vo..."
3,None,85,96,2,"Nova reitora da Universidade de Évora quer ""vo...",10 mai 2022
4,Non-claim,309,455,2,"Nova reitora da Universidade de Évora quer ""vo...",Temos que criar uma voz única na defesa dos in...
...,...,...,...,...,...,...
11698,Claim span,2492,2742,1536,Costa admite fundo ambiental e climático com 1...,este é ou o primeiro ou seguramente um dos pri...
11699,Claim object,2504,2579,1536,Costa admite fundo ambiental e climático com 1...,primeiro ou seguramente um dos primeiros instr...
11700,Claimer,2755,2784,1536,Costa admite fundo ambiental e climático com 1...,o primeiro-ministro português
11701,Non-claim,3021,3091,1536,Costa admite fundo ambiental e climático com 1...,Aqui está uma forma concreta de apoiar este me...


## Using Gemini with LangExtract


In [41]:
import langextract as lx
"""
HERE SHOULD BE THE KEY
"""
LANGEXTRACT_API_KEY="INSERT GEMINI KEY HERE!!!"
"""
HERE SHOULD BE THE KEY
"""
#doc_id= used as fewshot

def extract_info_gemini(input_text):
    example_text = 'Costa admite fundo ambiental e climático com 140 milhões de euros de dívida de Cabo Verde (C/ÁUDIO e VÍDEO)\n\n \nLisboa, 20 jun 2023 (Lusa) – O primeiro-ministro português admitiu hoje que a conversão da dívida de Cabo Verde a Portugal para integrar um fundo ambiental e climático poderá atingir 140 milhões de euros, mecanismo que considerou “um exemplo à escala global”.\nEsta posição foi transmitida por António Costa em São Bento, numa declaração conjunta com o seu homólogo cabo-verdiano, Ulisses Correia e Silva, após os dois países terem assinado um acordo para a integração de 12 milhões de euros de dívida de Cabo Verde num fundo ambiental e climático a ser criado pelo executivo da Praia.\n“A dívida que seria paga por Cabo Verde até 2025 - 12 milhões de euros - é integrada num fundo ambiental e climático para que o Estado cabo-verdiano apoie o financiamento e invista na transição climática no país. Em 2025, avaliaremos o sucesso desta operação. Estamos seguros que essa avaliação será positiva e poderemos alargar este mecanismo à restante dívida na totalidade da sua maturidade e no âmbito total, que são cerca de 140 milhões de euros”, declarou o líder do executivo português.\nEste acordo para a criação de um Fundo Climático Ambiental, envolvendo para já 12 milhões de euros, foi assinado nos jardins de São Bento pelo ministro das Finanças de Portugal, Fernando Medina, e pelo vice-primeiro-ministro e ministro das Finanças de Cabo Verde, Olavo Correia.\nSegundo o Governo português, os 12 milhões de euros correspondem ao montante a reembolsar pelo Estado cabo-verdiano, a título de capital, no âmbito do Contrato de Consolidação da Dívida de Cabo Verde a Portugal, celebrado em 1 de fevereiro de 2022.\n“É o primeiro passo de uma caminhada que queremos fazer em conjunto. Assinámos um documento particularmente inovador à escala global. Um acordo que permite conversão de dívida em investimento para acelerar a transição climática e promover as economias verde e azul”, defendeu António Costa, tendo ao seu lado Ulisses Correia e Silva, numa declaração sem direito a perguntas por parte dos jornalistas.\nAntónio Costa referiu depois que há várias conferências mundiais sobre o clima que se debate como financiar o investimento dos países em desenvolvimento para acelerar a sua transição para uma economia mais verde, num quadro “de uma aliança global para se enfrentar as alterações climáticas”.\n“Temos ouvido muitos discursos, temos assinado muitas declarações, mas creio que este é ou o primeiro ou seguramente um dos primeiros instrumentos financeiros concretos onde dois Estados acordam em que a dívida existente vai sendo convertida num fundo para financiar os investimentos necessários para acelerar a transição climática”, salientou o primeiro-ministro português.\nPerante o seu homólogo cabo-verdiano, António Costa sustentou ainda que o acordo agora assinado “é a tradução concreta de algo que todos têm consciência: Só em conjunto se poderá enfrentar o desafio global das alterações climáticas”.\n“Aqui está uma forma concreta de apoiar este mecanismo de financiamento”, insistiu, antes de observar que esta semana, em Paris, realiza-se uma conferência internacional sobre formas de financiamento da transição verde e azul à escala global.\n“Antes de essa conferência se iniciar, Portugal e Cabo Verde já deram um passo concreto. Estamos a concretizar agora algo que anunciámos já há alguns meses no Mindelo, permitindo acelerar a transição climática em Cabo Verde, através de um instrumento financeiro criativo e inovador”, acrescentou.\nAlém do acordo relativo à participação portuguesa no fundo ambiental e climático de Cabo Verde, os dois países assinaram também hoje um protocolo de cooperação sobre finanças públicas.\n \nPMF // JPS\nLusa/Fim'

  # Define extraction prompt
    prompt_description = ('Identifique claims (alegações factuais) e non-claims no texto, de acordo com as seguintes regras:'
                          'Claim:'
                          'Definição: Uma claim é uma afirmação factual, verificável e de interesse público, expressa em discurso direto (entre aspas), atribuída a alguém que não seja o jornalista.)'
                          'Segmento textual: Extraia apenas frases declarativas completas, com sentido próprio, sem incluir aspas ou ponto final.'
                          'Critérios de inclusão: '
                          'Claims geralmente aparecem ligadas a verbos de relato (afirmar, frisar, referir, disse, explicou). '
                          'Cada claim deve ser extraída individualmente, mesmo que esteja numa mesma citação.'
                          'Critérios de exclusão:'
                          'Não extrair frases incompletas ou sem sentido.'
                          'Não extrair frases informativas, de senso comum ou sem relevância pública.'
                          'Não extrair frases sobre possibilidades futuras ou hipóteses.'
                          'Non-Claim:'
                          'Definição: Frases subjetivas (opiniões, crenças, juízos pessoais), especulativas ou com referência a acontecimentos futuros não comprováveis.'
                          'Regras:'
                          'Devem ser frases declarativas completas em discurso direto (entre aspas).'
                          'Não incluir aspas nem ponto final.'
                          'Exclusão: frases narrativas do jornalista ou sem sentido completo.'
                          'Instruções Gerais:'
                          'Analise cada frase individualmente dentro de citações diretas.'
                          'Numa mesma citação podem existir claims e non-claims; classifique cada frase separadamente.')
    
      # Define example data with entities in order of appearance
    examples = [
          lx.data.ExampleData(
              text=example_text,
              extractions=[
                  lx.data.Extraction(extraction_class="claim", extraction_text="Temos ouvido muitos discursos, temos assinado muitas declarações, mas creio que este é ou o primeiro ou seguramente um dos primeiros instrumentos financeiros concretos onde dois Estados acordam em que a dívida existente vai sendo convertida num fundo para financiar os investimentos necessários para acelerar a transição climática"),
                  lx.data.Extraction(extraction_class="non-claim", extraction_text="Assinámos um documento particularmente inovador à escala global")
                
              ]
          )
    ]
    result = lx.extract(
        text_or_documents=input_text,
        prompt_description=prompt_description,
        examples=examples,
        api_key=LANGEXTRACT_API_KEY,
        model_id="gemini-2.5-flash",
    )
    
    return result


### Testing

In [42]:
# test example

test_text=data[5]["text"]
result=extract_info_gemini(test_text)

In [43]:
result

AnnotatedDocument(extractions=[Extraction(extraction_class='non-claim', extraction_text='A implicação da seca terá custos reais no nosso setor', char_interval=CharInterval(start_pos=340, end_pos=393), alignment_status=<AlignmentStatus.MATCH_EXACT: 'match_exact'>, extraction_index=1, group_index=0, description=None, attributes={}), Extraction(extraction_class='claim', extraction_text='Portugal ter o mais alto preço da energia e dos combustíveis na Europa', char_interval=CharInterval(start_pos=458, end_pos=528), alignment_status=<AlignmentStatus.MATCH_EXACT: 'match_exact'>, extraction_index=2, group_index=1, description=None, attributes={}), Extraction(extraction_class='non-claim', extraction_text='Com a seca, teremos de fazer o abastecimento através de autotanques', char_interval=CharInterval(start_pos=530, end_pos=597), alignment_status=<AlignmentStatus.MATCH_EXACT: 'match_exact'>, extraction_index=3, group_index=2, description=None, attributes={}), Extraction(extraction_class='non-cla

In [44]:
# Save the results to a JSONL file
lx.io.save_annotated_documents([result], output_name="extraction_results.jsonl", output_dir=".")

# Generate the visualization from the file
html_content = lx.visualize("extraction_results.jsonl")
with open("visualization.html", "w", encoding="utf-8") as f:
    if hasattr(html_content, 'data'):
        f.write(html_content.data)  # For Jupyter/Colab
    else:
        f.write(html_content)




LangExtract: Saving to extraction_results.jsonl: 1 docs [00:00, 1539.76 docs/s]

✓ Saved 1 documents to extraction_results.jsonl






LangExtract: Loading extraction_results.jsonl: 100%|██████████| 7.74k/7.74k [00:00<00:00, 27.6MB/s]

✓ Loaded 1 documents from extraction_results.jsonl


### Full data

In [48]:
import os
import time
import random
output_folder="results"
for entry in data:
  time.sleep(random.randint(60,90))
  print("processing",entry["doc_id"])
  try:
    result=extract_info_gemini(entry["text"])
  except Exception as e:
    time.sleep(120)
    result=extract_info_gemini(entry["text"])
  lx.io.save_annotated_documents([result], output_name=str(entry["doc_id"])+".json", output_dir=output_folder)
  html_content = lx.visualize(os.path.join(output_folder,str(entry["doc_id"])+".json"))
  with open(os.path.join(output_folder,str(entry["doc_id"])+".html"), "w") as f:
      if hasattr(html_content, 'data'):
          f.write(html_content.data)  # For Jupyter/Colab
      else:
          f.write(html_content)

  print("processed!!!",entry["doc_id"])



processing 1


InferenceRuntimeError: Parallel inference error: Gemini API error: 429 RESOURCE_EXHAUSTED. {'error': {'code': 429, 'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits.\n* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 250\nPlease retry in 37.731187414s.', 'status': 'RESOURCE_EXHAUSTED', 'details': [{'@type': 'type.googleapis.com/google.rpc.QuotaFailure', 'violations': [{'quotaMetric': 'generativelanguage.googleapis.com/generate_content_free_tier_requests', 'quotaId': 'GenerateRequestsPerDayPerProjectPerModel-FreeTier', 'quotaDimensions': {'model': 'gemini-2.5-flash', 'location': 'global'}, 'quotaValue': '250'}]}, {'@type': 'type.googleapis.com/google.rpc.Help', 'links': [{'description': 'Learn more about Gemini API quotas', 'url': 'https://ai.google.dev/gemini-api/docs/rate-limits'}]}, {'@type': 'ty